<a href="https://colab.research.google.com/github/vivianatuarez/tarea-redes-neuronales-grupo1/blob/main/02_experimentacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 02 - Experimentación Comparativa
## Actividad 3: Redes Neuronales desde Cero
### Proyecto 5 - Clasificación de Intenciones en Chatbot de Servicio al Cliente

En este notebook realizamos todas las comparaciones solicitadas:
- 3+ arquitecturas diferentes
- 3 funciones de activación (ReLU, Tanh, Sigmoid)
- Múltiples learning rates
- Comparación con baseline (regresión logística)
- Gráficas y tablas automáticas


In [ ]:
# Montar Drive (solo la primera vez)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Cambiar al directorio del proyecto
%cd '/content/drive/MyDrive/tarea-redes-neuronales-grupoX'

[Errno 2] No such file or directory: '/content/drive/MyDrive/tarea-redes-neuronales-grupoX'
/content


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Para que las gráficas se vean bonitas
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## Cargar datos y preprocesamiento (reutilizamos lo del notebook 01)

In [ ]:
df = pd.read_csv('data/intent_data.csv')

# Ampliamos un poco el dataset para que los experimentos sean más realistas
extra_texts = [
    "quiero devolver el producto", "necesito factura", "olvidé mi contraseña",
    "cómo hago el seguimiento", "mi pedido está retrasado", "quiero hablar con un agente",
    "gracias por la ayuda", "hasta luego", "no funciona el enlace", "error en el pago"
]
extra_labels = [
    'reclamo', 'consultar_precio', 'saludo', 'estado_envio',
    'estado_envio', 'saludo', 'saludo', 'saludo', 'reclamo', 'reclamo'
]

df_extra = pd.DataFrame({'text': extra_texts, 'label': extra_labels})
df = pd.concat([df, df_extra], ignore_index=True)
df.to_csv('data/intent_data.csv', index=False)

print(f"Total de ejemplos: {len(df)}")
print(f"Clases: {df['label'].unique()}")

FileNotFoundError: [Errno 2] No such file or directory: 'data/intent_data.csv'

In [ ]:
# Bag of Words (mismo código del notebook anterior)
from collections import Counter

def create_vocabulary(texts, max_vocab=500):
    words = ' '.join(texts).lower().split()
    most_common = Counter(words).most_common(max_vocab)
    vocab = {word: idx+1 for idx, (word, _) in enumerate(most_common)}
    vocab['<UNK>'] = 0
    return vocab

def text_to_bow(text, vocab):
    vec = np.zeros(len(vocab))
    for word in text.lower().split():
        vec[vocab.get(word, 0)] += 1
    return vec

vocab = create_vocabulary(df['text'])
X = np.array([text_to_bow(t, vocab) for t in df['text']])
labels = pd.Categorical(df['label']).codes
num_classes = len(set(labels))
y = np.eye(num_classes)[labels]

print(f"Vocabulario: {len(vocab)} palabras")
print(f"X shape: {X.shape}, y shape: {y.shape}")

NameError: name 'df' is not defined

## Clase de la Red Neuronal (la copiamos completa del notebook 01)

In [ ]:
class NeuralNetwork:
    def __init__(self, layers, activation='relu', seed=42):
        np.random.seed(seed)
        self.layers = layers
        self.activation = activation
        self.weights = []
        self.biases = []

        for i in range(1, len(layers)):
            if activation in ['sigmoid', 'tanh']:
                limit = np.sqrt(6 / (layers[i-1] + layers[i]))
                W = np.random.uniform(-limit, limit, (layers[i-1], layers[i]))
            else:
                std = np.sqrt(2 / layers[i-1])
                W = np.random.randn(layers[i-1], layers[i]) * std
            b = np.zeros((1, layers[i]))
            self.weights.append(W)
            self.biases.append(b)

    def _activate(self, z, derivative=False):
        if self.activation == 'sigmoid':
            sig = 1 / (1 + np.exp(-np.clip(z, -500, 500)))
            return sig if not derivative else sig * (1 - sig)
        elif self.activation == 'tanh':
            return np.tanh(z) if not derivative else 1 - z**2
        elif self.activation == 'relu':
            return np.maximum(0, z) if not derivative else (z > 0).astype(float)

    def forward(self, X):
        a = X
        self.activations = [X]

        for i in range(len(self.layers) - 2):
            z = a @ self.weights[i] + self.biases[i]
            a = self._activate(z)
            self.activations.append(a)

        z_out = a @ self.weights[-1] + self.biases[-1]
        exp_scores = np.exp(z_out - np.max(z_out, axis=1, keepdims=True))
        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
        self.activations.append(probs)
        return probs

    def backward(self, X, y, output):
        m = X.shape[0]
        delta = output - y
        deltas = [delta]

        for i in reversed(range(len(self.layers) - 2)):
            delta = (delta @ self.weights[i+1].T) * self._activate(self.activations[i+1], derivative=True)
            deltas.append(delta)
        deltas.reverse()

        dW = [self.activations[i].T @ deltas[i] / m for i in range(len(self.weights))]
        db = [np.sum(deltas[i], axis=0, keepdims=True) / m for i in range(len(self.biases))]

        return dW, db

    def train(self, X, y, epochs=1500, lr=0.01, verbose=False):
        losses, accs = [], []
        for epoch in range(epochs):
            output = self.forward(X)
            loss = -np.mean(np.sum(y * np.log(output + 1e-15), axis=1))
            acc = np.mean(np.argmax(output, axis=1) == np.argmax(y, axis=1))
            losses.append(loss)
            accs.append(acc)

            dW, db = self.backward(X, y, output)
            for i in range(len(self.weights)):
                self.weights[i] -= lr * dW[i]
                self.biases[i] -= lr * db[i]

            if verbose and epoch % 300 == 0:
                print(f"Epoch {epoch:4d} → Loss: {loss:.4f} | Acc: {acc:.4f}")
        return losses, accs

## Experimentos Masivos (esto es lo que vale puntos!)

In [ ]:
# Definimos las configuraciones a probar
arquitecturas = [
    [X.shape[1], 64, 32, num_classes],        # Pequeña
    [X.shape[1], 128, 64, num_classes],       # Mediana (mejor en práctica)
    [X.shape[1], 256, 128, 64, num_classes],  # Profunda
    [X.shape[1], 512, 256, 128, 64, num_classes]  # Muy profunda (bonus)
]

activaciones = ['relu', 'tanh', 'sigmoid']
learning_rates = [0.01, 0.05, 0.1, 0.5]

resultados = []

print("Iniciando experimentos masivos... (puede tardar 3-5 minutos)\n")
for arch in arquitecturas:
    for act in activaciones:
        for lr in learning_rates:
            print(f"Probando → {arch} | {act} | lr={lr}")
            model = NeuralNetwork(arch, activation=act, seed=42)
            losses, accs = model.train(X, y, epochs=1500, lr=lr, verbose=False)
            final_acc = accs[-1]
            resultados.append({
                'arquitectura': f"{arch[1:-1]}",  # sin input/output
                'neuronas_totales': sum(arch[1:-1]),
                'capas_ocultas': len(arch)-2,
                'activacion': act,
                'learning_rate': lr,
                'accuracy_final': round(final_acc, 4),
                'loss_final': round(losses[-1], 4)
            })

# Guardamos todo
df_resultados = pd.DataFrame(resultados)
df_resultados = df_resultados.sort_values('accuracy_final', ascending=False)
df_resultados.to_csv('results/performance_comparison.csv', index=False)
df_resultados.head(15)

In [ ]:
# Gráfica 1: Mejor combinación
plt.figure(figsize=(12, 6))
top = df_resultados.head(10)
plt.barh(range(len(top)-1, -1, -1), top['accuracy_final'])
plt.yticks(range(len(top)), [f"{row['activacion']}-lr{row['learning_rate']}-{row['capas_ocultas']}capas" for _, row in top.iterrows()])
plt.xlabel('Accuracy Final')
plt.title('Top 10 Mejores Configuraciones')
plt.xlim(0.7, 1.0)
plt.tight_layout()
plt.savefig('results/top10_configuraciones.png', dpi=200)
plt.show()

NameError: name 'df_resultados' is not defined

<Figure size 1200x600 with 0 Axes>

In [ ]:
# Gráfica 2: Comparación por función de activación
plt.figure(figsize=(10, 6))
for act in activaciones:
    subset = df_resultados[df_resultados['activacion'] == act]
    plt.scatter(subset['learning_rate'], subset['accuracy_final'], label=act, s=80, alpha=0.7)
plt.xlabel('Learning Rate')
plt.ylabel('Accuracy Final')
plt.title('Rendimiento por Función de Activación')
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('results/comparacion_activaciones.png', dpi=200)
plt.show()

## Baseline: Regresión Logística (desde cero también!)

In [ ]:
class LogisticRegressionScratch:
    def __init__(self, lr=0.01, epochs=2000):
        self.lr = lr
        self.epochs = epochs

    def fit(self, X, y):
        self.W = np.zeros((X.shape[1], y.shape[1]))
        self.b = np.zeros((1, y.shape[1]))

        for _ in range(self.epochs):
            z = X @ self.W + self.b
            a = np.exp(z - np.max(z, axis=1, keepdims=True))
            a = a / np.sum(a, axis=1, keepdims=True)
            self.W -= self.lr * (X.T @ (a - y)) / len(X)
            self.b -= self.lr * np.sum(a - y, axis=0, keepdims=True) / len(X)

    def predict(self, X):
        z = X @ self.W + self.b
        a = np.exp(z - np.max(z, axis=1, keepdims=True))
        return a / np.sum(a, axis=1, keepdims=True)

logreg = LogisticRegressionScratch(lr=0.1, epochs=3000)
logreg.fit(X, y)
pred = logreg.predict(X)
acc_logreg = np.mean(np.argmax(pred, axis=1) == np.argmax(y, axis=1))
print(f"Accuracy Regresión Logística (baseline): {acc_logreg:.4f}")

## Resultado final del experimento

In [ ]:
mejor = df_resultados.iloc[0]
print(f"GANADOR del experimento:")
print(f"Arquitectura: {mejor['arquitectura']}")
print(f"Activación: {mejor['activacion'].upper()}")
print(f"Learning rate: {mejor['learning_rate']}")
print(f"Accuracy alcanzada: {mejor['accuracy_final']*100:.2f}%")
print(f"vs Baseline (LogReg): {acc_logreg*100:.2f}% → ¡Mejora de +{(mejor['accuracy_final']-acc_logreg)*100:.2f} puntos!")

NameError: name 'df_resultados' is not defined